# Devoir 3, Question 2 : Discriminants non-linéaires

# Homework 3, Question 2: Nonlinear discriminants

## Code préambule

## Preamble code

In [1]:
import numpy
import requests
import time
import pandas
pandas.set_option('display.max_colwidth', 0)
import collections
import itertools

from IPython import display

from io import BytesIO
from http.client import HTTPConnection
from sklearn.model_selection import KFold, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import minmax_scale
from sklearn.svm import SVC

# Nous ne voulons pas être signalé par ce type d'avertissement, non pertinent pour le devoir
# We don't want to be signaled of this warning, irrelevant for the homework
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Fonction pour vérifier le temps d'exécution
# Function to verify execution time
_times = []
def checkTime(maxduration, question):
    duration = _times[-1] - _times[-2]
    if duration > maxduration:
        print("[ATTENTION] Votre code pour la question {0} met trop de temps à s'exécuter! ".format(question)+
              "Le temps maximum permis est de {0:.4f} secondes, ".format(maxduration)+
              "mais votre code a requis {0:.4f} secondes! ".format(duration)+
              "Assurez-vous que vous ne faites pas d'appels bloquants (par exemple à show()) dans cette boucle!")

# Fonction pour obtenir jeu de données Pendigits, ne pas modifier
# Function to obtain the Pendigits dataset, do not modify
def fetchPendigits(file_id):
    """
    Cette fonction télécharge le jeu de données pendigits et le
    retourne sous forme de deux tableaux numpy. Le premier élément
    retourné par cette fonction est un tableau de 10992x16 qui
    contient les données des caractères; le second élément est un
    vecteur de 10992 qui contient les valeurs cible (étiquettes).
    This function download dataset pendigits and return it as numpy
    arrays. The first element returned is a 10992x16 array with the
    characters data; the second element is a vector of size 10992 that
    contains the targets (labels).
    """
    URL = "https://drive.google.com/uc?"
    session = requests.Session()
    
    r = session.get(URL, params = { 'id' : file_id }, stream = True)
    error_msg = f'ERREUR: impossible de télécharger UCI Pendigits (code={r.status_code})'
    assert(r.status_code == 200), error_msg
    
    params = { 'id' : file_id, 'confirm' : 'download_warning' }
    r = session.get(URL, params = params, stream = True)
    stream = BytesIO(r.content)
    dataPendigits = numpy.load(stream)
    return dataPendigits[:, :-1].astype('float32'), dataPendigits[:, -1]

# Définition des durées d'exécution maximales pour chaque classifieur
# Definition of maximum execution time for each classifier
TMAX_KNN = 40
TMAX_SVM = 200
TMAX_MLP = 400
TMAX_EVAL = 80
DRIVE_ID = '1OxSkSF2RLDUcAyFd1MpG3RYBHOX8ni4V'

## Q2A
Soit les classifieurs suivants, présentés en classe :
- Classement par les $k$-plus proches voisins avec distance euclidienne (`neighbors.KNeighborsClassifier`);
- Séparateurs à vastes marges avec noyau gaussien (`svm.SVC`), notez que le paramètre du noyau de la fonction est $\gamma=\frac{1}{2\sigma^2}$;
- Perceptron multicouche pour le classement (`neural_network.MLPClassifier`). Pour cette méthode, utilisez le paramètre `max_iter=100`.

Pour les trois algorithmes présentés ci-haut, déterminez les hyperparamètres importants qui peuvent influencer significativement les résultats d’apprentissage (limitez-vous à un maximum de deux hyperparamètres importants par algorithme). Dans le tableau de réponse, affichez les deux hyperparamètres choisis. Donnez également la plage de valeurs qui devrait être testée pour chacun de ces hyperparamètres afin de faire un bon ajustement de la configuration des classifieurs sur le jeu Pendigits. Vous testerez ces plages dans un prochain exercice.

Le jeu de données [Pendigits](https://archive.ics.uci.edu/ml/datasets/Pen-Based+Recognition+of+Handwritten+Digits) vise à reconnaître des chiffres manuscrits (10 classes) à partir des coordonnées de 8 points des tracés des caractères (16 variables).

## Q2A
Consider the following classifiers, presented in class:
- Classification by $k$-nearest neighbors with Euclidean distance (`neighbors.KNeighborsClassifier`);
- Support vector machine with Gaussian kernel (`svm.SVC`), note that the kernel parameter of the function is $\gamma=\frac{1}{2\sigma^2}$;
- Multilayer perceptron for classification (`neural_network.MLPClassifier`). For this method, use the parameter `max_iter = 100`.

For the three algorithms presented above, determine the important hyperparameters that can significantly influence learning outcomes (limit yourself to a maximum of two important hyperparameters per algorithm). In the answer table, display the 2 chosen hyperparameters. Also give the range of values that should be tested for each of these hyperparameters in order to make a good fit for the configuration of the classifiers on the Pendigits game. You will test these ranges in a future exercise.

The [Pendigits](https://archive.ics.uci.edu/ml/datasets/Pen-Based+Recognition+of+Handwritten+Digits) dataset aims at recognizing handwritten digits (10 classes) from the coordinated of 8 points taken from the character plots (16 variables).

### Entrez votre solution à Q2A dans la cellule ci-dessous
Rapportez les paramètres d’entraînement et les plages de valeurs correspondantes que vous avez choisis pour chacun des algorithmes, en modifiant le tableau donné ci-dessous, dans la cellule de solution.
~~~md
Algorithme | Parametre 1    | Parametre 2    | Plage param. 1 |Plage param. 2 |
-----------|----------------|----------------|----------------|---------------|
KNN        | -              | -              | -              | -             |
SVM        | -              | -              | -              | -             |
MLP        | -              | -              | -              | -             |
~~~

### Enter your answer to Q2A in the cell below
Report the training parameters and the corresponding ranges of values you have chosen for each of the algorithms, by modifying the table structure below into the solution cell.
~~~md
Algorithme | Parametre 1    | Parametre 2    | Plage param. 1 |Plage param. 2 |
-----------|----------------|----------------|----------------|---------------|
KNN        | -              | -              | -              | -             |
SVM        | -              | -              | -              | -             |
MLP        | -              | -              | -              | -             |
~~~


        <div class="feedback-cell" style="background: rgba(100 , 100 , 100 , 0.4)">
            <h3>Votre soumission a été enregistrée!</h3>
            <ul>
                <li>notez qu'il n'y a <strong>pas</strong> de correction automatique pour cet exercice&puncsp;;</li>
                <li>par conséquent, votre note est <strong>actuellement</strong> zéro&puncsp;;</li>
                <li>elle sera cependant ajustée par le professeur dès que la correction manuelle sera complétée&puncsp;;</li>
                <li>vous pouvez soumettre autant de fois que nécessaire jusqu'à la date d'échéance&puncsp;;</li>
                <li>mais évitez de soumettre inutilement.</li>
            </ul>
        </div>
        

Algorithme | Parametre 1    | Parametre 2    | Plage param. 1 |Plage param. 2 |
-----------|----------------|----------------|----------------|---------------|
KNN        | n_neighbors    | weights        | 1,3,5,7,11,13,15,25,35,45 | 'uniform', 'distance' |
SVM        | C              | gamma          | 10^-5,10^-4,...,10^5 | 0.01,0.1,1,10|
MLP        | hidden_layer_sizes | alpha      | (15,),(20,),(25,),(50,),(75,) | 2^0*10^-4,..., 2^6*10-4 |

## Q2B
Tester les discriminants de la question Q2A avec le jeu de données Pendigits. Utilisez la fonction `fetchPendigits` fournie en préambule. Normalisez les données dans $[0,1]^D$ avant vos traitements (`preprocessing.minmax_scale`). Utilisez 5500 instances choisies au hasard comme données d’entraînement et les 5492 données restantes pour les tests. Notez que la dernière colonne du jeu de données correspond aux étiquettes de classe.

## Q2B
Test the discriminants of question Q2A with the Pendigits dataset. Use the `fetchPendigits` function provided in the preamble. Normalize the data in $[0,1]^D$ before your processing (`preprocessing.minmax_scale`). Use 5500 randomly selected instances for training data and the remaining 5492 data for testing. Note that the last column of the dataset corresponds to the class labels.

### Patron de code réponse à l'exercice Q2B

### Q2B answer code template

In [3]:
# Initialisation du jeu de données Pendigits
# Initializing Pendigits dataset
print('Téléchargement du jeu de données Pendigits...')
data, target = fetchPendigits(DRIVE_ID)
print('Téléchargement terminé.')

# *** TODO ***
# Normalisez les données d'entrée entre 0 et 1 pour toutes les dimensions.
# Notez que la fonction retourne les données d'une manière différente des
# fonctions load_* que vous avez déjà utilisées (ex. load_iris(), 
# load_breast_cancer()). Assurez-vous que vous utilisez correctement les
# données et qu'elles sont du bon type!
# Normalize the input data between 0 and 1 for all dimensions.
# Note that the function returns the data in a different way than the load_* 
# functions you you have used before (e.g. load_iris(), load_breast_cancer()).
# Make sure you are using the data correctly and that it is of the right type!

# Séparez le jeu de données Pendigits en deux sous-jeux: entraînement (5500) 
# et test (reste des données). Pour la suite du code, rappelez-vous que vous ne
# pouvez PAS vous servir du jeu de test pour déterminer la configuration
# d'hyper-paramètres la plus performante. Ce jeu de test ne doit être utilisé qu'à
# la toute fin, pour rapporter les résultats finaux en généralisation.
# Split the Pendigits dataset into two subsets: training (5500) and testing
# (the rest of the data). For the remainder of the code, remember that you
# cannot use the test set to determine the most appropriate hyper-parameter
# configuration. This test set should only be used at the very end, to report the
# final results in generalization.
# ******

# Création du tableau pour accumuler les résultats
# Creation of the table to accumulate results
results = {'Classifiers':[],
           'Optimal_hp1':[],
           'Optimal_hp2':[],
           'Time_train':[],
           'Time_test':[],
           'Score_train':[],
           'Score_test':[],
          }

_times.append(time.time())

# Pour chaque classifieurs (k-PPV, SVM à noyau gaussien, Perceptron multicouche),
# déterminez les valeurs optimales des hyperparamètres à utiliser. Suivez les
# instructions de l'énoncé de Q2A et Q2B quant au nombre d'hyperparamètres à optimiser.
# Vous êtes libres d'utiliser la méthodologie que vous souhaitez, en autant que
# vous ne touchez pas au jeu de test pour l'optimisation.
# For each classifier (k-NN, Gaussian kernel SVM, Multi-layer Perceptron),
# determine the optimal values of the hyperparameters to use. Follow the
# instructions in the statement of Q2A and Q2B regarding the number of
# hyperparameters to optimize. You are free to use any methodology you wish, as
# long as you do not make use of the test set for optimization.

# -------

# Optimisez la paramétrisation du k-PPV
# Optimize k-NN parametrization

# *** TODO ***
# Ajustez les hyperparamètres du k-PPV
# Adjust k-NN hyperparamters
clf_knn = KNeighborsClassifier()
# ******

clf_name = clf_knn.__class__.__name__
if clf_name not in results['Classifiers']:
    results['Classifiers'].append(clf_name)

# *** TODO ***   
# Calculez le temps d'entraînement pour le k-PPV
# et mettez-le dans la variable time_train en
# remplaçant le 0.
# Calculate the training time for the k-NN
# and put it in the time_train variable by
# replacing the 0.
time_train = 0
# ******

results['Time_train'].append(time_train)

# *** TODO ***
# Calculez le score d'entraînement pour le k-PPV
# et mettez-le dans la variable score_train en
# remplaçant le 0.
# Calculate the training score for the k-NN
# and put it in the score_train variable by
# replacing the 0.
score_train = 0
# ******

results['Score_train'].append(score_train)

# *** TODO ***
# Stockez vos valeurs optimales 
# d'hyperparamètres dans les variables
# optimal_hp1 et optimal_hp2 en remplaçant
# le 0.
# Store your optimal hyperparameters values
# in the variables optimal_hp1 and optimal_hp2 by
# replacing the 0.
optimal_hp1 = 0
optimal_hp2 = 0
# ******

results['Optimal_hp1'].append(optimal_hp1)
results['Optimal_hp2'].append(optimal_hp2)
_times.append(time.time())
checkTime(TMAX_KNN, "k-plus proches voisins")

# -------

# Optimisez la paramétrisation du SVM à noyau gaussien
# Optimize Gaussian kernel SVM parametrization

# *** TODO ***
# Ajustez les hyperparamètres du SVM
# Adjust SVM hyperparamters
clf_svc = SVC()
# ******

clf_name = clf_svc.__class__.__name__
if clf_name not in results['Classifiers']:
    results['Classifiers'].append(clf_name)

# *** TODO ***   
# Calculez le temps d'entraînement pour le SVM
# et mettez-le dans la variable time_train en
# remplaçant le 0.
# Calculate the training time for the SVM
# and put it in the time_train variable by
# replacing the 0.
time_train = 0
# ******

results['Time_train'].append(time_train)

# *** TODO ***
# Calculez le score d'entraînement pour le SVM
# et mettez-le dans la variable score_train en
# remplaçant le 0.
# Calculate the training score for the SVM
# and put it in the score_train variable by
# replacing the 0.
score_train = 0
# ******

results['Score_train'].append(score_train)

# *** TODO ***
# Stockez vos valeurs optimales 
# d'hyperparamètres dans les variables
# optimal_hp1 et optimal_hp2 en remplaçant
# le 0.
# Store your optimal hyperparameters values
# in the variables optimal_hp1 and optimal_hp2 by
# replacing the 0.
optimal_hp1 = 0
optimal_hp2 = 0
# ******

results['Optimal_hp1'].append(optimal_hp1)
results['Optimal_hp2'].append(optimal_hp2)
_times.append(time.time())
checkTime(TMAX_SVM, "Support vector machine")

# -------

# Optimisez la paramétrisation du perceptron multicouche
# Optimize multilayer perceptron parametrization

# *** TODO ***
# Ajustez les hyperparamètres du MLP
# Adjust MLP hyperparamters
clf_mlp = MLPClassifier()
# ******

clf_name = clf_mlp.__class__.__name__
if clf_name not in results['Classifiers']:
    results['Classifiers'].append(clf_name)

# *** TODO ***   
# Calculez le temps d'entraînement pour le PMC
# et mettez-le dans la variable time_train en
# remplaçant le 0.
# Calculate the training time for the MLP
# and put it in the time_train variable by
# replacing the 0.
time_train = 0
# ******

results['Time_train'].append(time_train)

# *** TODO ***
# Calculez le score d'entraînement pour le PMC
# et mettez-le dans la variable score_train en
# remplaçant le 0.
# Calculate the training score for the MLP
# and put it in the score_train variable by
# replacing the 0.
score_train = 0
# ******

results['Score_train'].append(score_train)

# *** TODO ***
# Stockez vos valeurs optimales 
# d'hyperparamètres dans les variables
# optimal_hp1 et optimal_hp2 en remplaçant
# le 0.
# Store your optimal hyperparameters values
# in the variables optimal_hp1 and optimal_hp2 by
# replacing the 0.
optimal_hp1 = 0
optimal_hp2 = 0
# ******

results['Optimal_hp1'].append(optimal_hp1)
results['Optimal_hp2'].append(optimal_hp2)
_times.append(time.time())
checkTime(TMAX_MLP, "Perceptron multicouche")

# -------

# Après avoir bien recherché les meilleures paramétrisations
# pour vos discriminants, évaluez leurs performances sur le
# jeu de test. Rapportez les performances dans le tableau généré.

# k-PPV / k-NN

# *** TODO ***   
# Calculez le temps d'inference pour le k-PPV
# et mettez-le dans la variable time_test en
# remplaçant le 0.
# Calculate the inference time for the k-NN
# and put it in the time_test variable by
# replacing the 0.
time_test = 0

# Calculez le score en test pour le k-PPV
# et mettez-le dans la variable score_test en
# remplaçant le 0.
# Calculate the test score for the k-NN
# and put it in the score_test variable by
# replacing the 0.
score_test = 0
# ******

results['Time_test'].append(time_test)
results['Score_test'].append(score_test)

# SVM

# *** TODO ***   
# Calculez le temps d'inference pour le SVM
# et mettez-le dans la variable time_test en
# remplaçant le 0.
# Calculate the inference time for the SVM
# and put it in the time_test variable by
# replacing the 0.
time_test = 0

# Calculez le score en test pour le SVM
# et mettez-le dans la variable score_test en
# remplaçant le 0.
# Calculate the test score for the SVM
# and put it in the score_test variable by
# replacing the 0.
score_test = 0
# ******

results['Time_test'].append(time_test)
results['Score_test'].append(score_test)

# PMC / MLP

# *** TODO ***   
# Calculez le temps d'inference pour le PMC
# et mettez-le dans la variable time_test en
# remplaçant le 0.
# Calculate the inference time for the MLP
# and put it in the time_test variable by
# replacing the 0.
time_test = 0

# Calculez le score en test pour le PMC
# et mettez-le dans la variable score_test en
# remplaçant le 0.
# Calculate the test score for the MLP
# and put it in the score_test variable by
# replacing the 0.
score_test = 0
# ******

results['Time_test'].append(time_test)
results['Score_test'].append(score_test)

_times.append(time.time())
checkTime(TMAX_EVAL, "Evaluation des modèles")

# Affichage des résultats
# Display results
df = pandas.DataFrame(results)
display.display(df)

Téléchargement du jeu de données Pendigits...
Téléchargement terminé.


,Classifiers,Optimal_hp1,Optimal_hp2,Time_train,Time_test,Score_train,Score_test
0,KNeighborsClassifier,0,0,0,0,0,0
1,SVC,0,0,0,0,0,0
2,MLPClassifier,0,0,0,0,0,0


### Entrez votre solution à Q2B dans la cellule ci-dessous

### Enter your answer to Q2B in the cell below

In [4]:
# Initialisation du jeu de données Pendigits
# Initializing Pendigits dataset
print('Téléchargement du jeu de données Pendigits...')
data, target = fetchPendigits(DRIVE_ID)
print('Téléchargement terminé.')

# *** TODO ***
# Normalisez les données d'entrée entre 0 et 1 pour toutes les dimensions.
# Notez que la fonction retourne les données d'une manière différente des
# fonctions load_* que vous avez déjà utilisées (ex. load_iris(), 
# load_breast_cancer()). Assurez-vous que vous utilisez correctement les
# données et qu'elles sont du bon type!
# Normalize the input data between 0 and 1 for all dimensions.
# Note that the function returns the data in a different way than the load_* 
# functions you you have used before (e.g. load_iris(), load_breast_cancer()).
# Make sure you are using the data correctly and that it is of the right type!
data = minmax_scale(data.astype("float64"))

# Séparez le jeu de données Pendigits en deux sous-jeux: entraînement (5500) 
# et test (reste des données). Pour la suite du code, rappelez-vous que vous ne
# pouvez PAS vous servir du jeu de test pour déterminer la configuration
# d'hyper-paramètres la plus performante. Ce jeu de test ne doit être utilisé qu'à
# la toute fin, pour rapporter les résultats finaux en généralisation.
# Split the Pendigits dataset into two subsets: training (5500) and testing
# (the rest of the data). For the remainder of the code, remember that you
# cannot use the test set to determine the most appropriate hyper-parameter
# configuration. This test set should only be used at the very end, to report the
# final results in generalization.
# ******
X1, X2, y1, y2 = train_test_split(data, target, train_size=5500, random_state=648)

# Création du tableau pour accumuler les résultats
# Creation of the table to accumulate results
results = {'Classifiers':[],
           'Optimal_hp1':[],
           'Optimal_hp2':[],
           'Time_train':[],
           'Time_test':[],
           'Score_train':[],
           'Score_test':[],
          }

_times.append(time.time())

# Pour chaque classifieurs (k-PPV, SVM à noyau gaussien, Perceptron multicouche),
# déterminez les valeurs optimales des hyperparamètres à utiliser. Suivez les
# instructions de l'énoncé de Q2A et Q2B quant au nombre d'hyperparamètres à optimiser.
# Vous êtes libres d'utiliser la méthodologie que vous souhaitez, en autant que
# vous ne touchez pas au jeu de test pour l'optimisation.
# For each classifier (k-NN, Gaussian kernel SVM, Multi-layer Perceptron),
# determine the optimal values of the hyperparameters to use. Follow the
# instructions in the statement of Q2A and Q2B regarding the number of
# hyperparameters to optimize. You are free to use any methodology you wish, as
# long as you do not make use of the test set for optimization.
kf = KFold(n_splits=2)

# -------

# Optimisez la paramétrisation du k-PPV
# Optimize k-NN parametrization
my_weights = ['uniform', 'distance']
my_neighbors = [1, 3, 5, 7, 11, 13, 15, 25, 35, 45]

t1 = time.time()
knn_score = -1
hp1, hp2 = "", 0
knn = None

for w in my_weights:
    for n in my_neighbors:
        clf = KNeighborsClassifier(n_neighbors=n, weights=w)
        scores = numpy.zeros(kf.get_n_splits(X1))
        count = 0
        for train_index, test_index in kf.split(X1):
            X_train, X_test = X1[train_index], X1[test_index]
            y_train, y_test = y1[train_index], y1[test_index]
            clf.fit(X_train, y_train)
            scores[count] = clf.score(X_test, y_test)
            count = count + 1
        tmp_score = numpy.mean(scores)
        if tmp_score > knn_score:
            knn_score = tmp_score
            hp1, hp2 = w, n
            knn = clf

t2 = time.time()

# *** TODO ***
# Ajustez les hyperparamètres du k-PPV
# Adjust k-NN hyperparamters
clf_knn = KNeighborsClassifier()
# ******

clf_name = clf_knn.__class__.__name__
if clf_name not in results['Classifiers']:
    results['Classifiers'].append(clf_name)

# *** TODO ***   
# Calculez le temps d'entraînement pour le k-PPV
# et mettez-le dans la variable time_train en
# remplaçant le 0.
# Calculate the training time for the k-NN
# and put it in the time_train variable by
# replacing the 0.
time_train = t2 - t1
# ******

results['Time_train'].append(time_train)

# *** TODO ***
# Calculez le score d'entraînement pour le k-PPV
# et mettez-le dans la variable score_train en
# remplaçant le 0.
# Calculate the training score for the k-NN
# and put it in the score_train variable by
# replacing the 0.
score_train = knn_score
# ******

results['Score_train'].append(score_train)

# *** TODO ***
# Stockez vos valeurs optimales 
# d'hyperparamètres dans les variables
# optimal_hp1 et optimal_hp2 en remplaçant
# le 0.
# Store your optimal hyperparameters values
# in the variables optimal_hp1 and optimal_hp2 by
# replacing the 0.
optimal_hp1 = hp1
optimal_hp2 = hp2

# ******
results['Optimal_hp1'].append(optimal_hp1)
results['Optimal_hp2'].append(optimal_hp2)
_times.append(time.time())
checkTime(TMAX_KNN, "k-plus proches voisins")

# -------

# Optimisez la paramétrisation du SVM à noyau gaussien
# Optimize Gaussian kernel SVM parametrization
my_Cs = [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]
my_gammas = [0.01, 0.1 ,1 , 10]

t1 = time.time()
svm_score = -1
hp1, hp2 = "", 0
svm = None


for c in my_Cs:
    for g in my_gammas:
        clf = SVC(C=c, gamma=g)
        scores = numpy.zeros(kf.get_n_splits(X1))
        count = 0
        for train_index, test_index in kf.split(X1):
            X_train, X_test = X1[train_index], X1[test_index]
            y_train, y_test = y1[train_index], y1[test_index]
            clf.fit(X_train, y_train)
            scores[count] = clf.score(X_test, y_test)
            count = count + 1
        tmp_score = numpy.mean(scores)
        if tmp_score > svm_score:
            svm_score = tmp_score
            hp1, hp2 = c, g
            svm = clf

t2 = time.time()
            
# *** TODO ***
# Ajustez les hyperparamètres du SVM
# Adjust SVM hyperparamters
clf_svc = SVC()
# ******

clf_name = clf_svc.__class__.__name__
if clf_name not in results['Classifiers']:
    results['Classifiers'].append(clf_name)

# *** TODO ***   
# Calculez le temps d'entraînement pour le SVM
# et mettez-le dans la variable time_train en
# remplaçant le 0.
# Calculate the training time for the SVM
# and put it in the time_train variable by
# replacing the 0.
time_train = t2 - t1
# ******

results['Time_train'].append(time_train)

# *** TODO ***
# Calculez le score d'entraînement pour le SVM
# et mettez-le dans la variable score_train en
# remplaçant le 0.
# Calculate the training score for the SVM
# and put it in the score_train variable by
# replacing the 0.
score_train = svm_score
# ******

results['Score_train'].append(score_train)

# *** TODO ***
# Stockez vos valeurs optimales 
# d'hyperparamètres dans les variables
# optimal_hp1 et optimal_hp2 en remplaçant
# le 0.
# Store your optimal hyperparameters values
# in the variables optimal_hp1 and optimal_hp2 by
# replacing the 0.
optimal_hp1 = hp1
optimal_hp2 = hp2
# ******

results['Optimal_hp1'].append(optimal_hp1)
results['Optimal_hp2'].append(optimal_hp2)
_times.append(time.time())
checkTime(TMAX_SVM, "Support vector machine")

# -------

# Optimisez la paramétrisation du perceptron multicouche
# Optimize multilayer perceptron parametrization
my_alphas = [0.00001, 0.00002, 0.00004, 0.00008, 0.00016, 0.00032, 0.00064] 
my_hidden_layer_sizes = [(15,), (20,), (25,), (50,), (75,)]

t1 = time.time()

mlp_score = -1
hp1, hp2 = "", 0
mlp = None

for hls in my_hidden_layer_sizes:
    for a in my_alphas:
        clf = MLPClassifier(hidden_layer_sizes=hls, alpha=a, max_iter=100)
        scores = numpy.zeros(kf.get_n_splits(X1))
        count = 0
        for train_index, test_index in kf.split(X1):
            X_train, X_test = X1[train_index], X1[test_index]
            y_train, y_test = y1[train_index], y1[test_index]
            clf.fit(X_train, y_train)
            scores[count] = clf.score(X_test, y_test)
            count = count + 1
        tmp_score = numpy.mean(scores)
        if tmp_score > mlp_score:
            mlp_score = tmp_score
            hp1, hp2 = hls, a
            mlp = clf
            
t2 = time.time()

# *** TODO ***
# Ajustez les hyperparamètres du MLP
# Adjust MLP hyperparamters
clf_mlp = MLPClassifier()
# ******

clf_name = clf_mlp.__class__.__name__
if clf_name not in results['Classifiers']:
    results['Classifiers'].append(clf_name)

# *** TODO ***   
# Calculez le temps d'entraînement pour le PMC
# et mettez-le dans la variable time_train en
# remplaçant le 0.
# Calculate the training time for the MLP
# and put it in the time_train variable by
# replacing the 0.
time_train = t2 - t1
# ******

results['Time_train'].append(time_train)

# *** TODO ***
# Calculez le score d'entraînement pour le PMC
# et mettez-le dans la variable score_train en
# remplaçant le 0.
# Calculate the training score for the MLP
# and put it in the score_train variable by
# replacing the 0.
score_train = mlp_score
# ******

results['Score_train'].append(score_train)

# *** TODO ***
# Stockez vos valeurs optimales 
# d'hyperparamètres dans les variables
# optimal_hp1 et optimal_hp2 en remplaçant
# le 0.
# Store your optimal hyperparameters values
# in the variables optimal_hp1 and optimal_hp2 by
# replacing the 0.
optimal_hp1 = hp1
optimal_hp2 = hp2
# ******

results['Optimal_hp1'].append(optimal_hp1)
results['Optimal_hp2'].append(optimal_hp2)
_times.append(time.time())
checkTime(TMAX_MLP, "Perceptron multicouche")

# -------

# Après avoir bien recherché les meilleures paramétrisations
# pour vos discriminants, évaluez leurs performances sur le
# jeu de test. Rapportez les performances dans le tableau généré.

# k-PPV / k-NN
t1 = time.time()

knn_test_score = knn.score(X2, y2)

t2 = time.time()

# *** TODO ***   
# Calculez le temps d'inference pour le k-PPV
# et mettez-le dans la variable time_test en
# remplaçant le 0.
# Calculate the inference time for the k-NN
# and put it in the time_test variable by
# replacing the 0.
time_test = t2 - t1

# Calculez le score en test pour le k-PPV
# et mettez-le dans la variable score_test en
# remplaçant le 0.
# Calculate the test score for the k-NN
# and put it in the score_test variable by
# replacing the 0.
score_test = knn_test_score
# ******

results['Time_test'].append(time_test)
results['Score_test'].append(score_test)

# SVM
t1 = time.time()

svm_test_score = svm.score(X2, y2)

t2 = time.time()


# *** TODO ***   
# Calculez le temps d'inference pour le SVM
# et mettez-le dans la variable time_test en
# remplaçant le 0.
# Calculate the inference time for the SVM
# and put it in the time_test variable by
# replacing the 0.
time_test = t2 - t1

# Calculez le score en test pour le SVM
# et mettez-le dans la variable score_test en
# remplaçant le 0.
# Calculate the test score for the SVM
# and put it in the score_test variable by
# replacing the 0.
score_test = svm_test_score
# ******

results['Time_test'].append(time_test)
results['Score_test'].append(score_test)

# PMC / MLP
t1 = time.time()

mlp_test_score = mlp.score(X2, y2)

t2 = time.time()

# *** TODO ***   
# Calculez le temps d'inference pour le PMC
# et mettez-le dans la variable time_test en
# remplaçant le 0.
# Calculate the inference time for the MLP
# and put it in the time_test variable by
# replacing the 0.
time_test = t2 - t1

# Calculez le score en test pour le PMC
# et mettez-le dans la variable score_test en
# remplaçant le 0.
# Calculate the test score for the MLP
# and put it in the score_test variable by
# replacing the 0.
score_test = mlp_test_score
# ******

results['Time_test'].append(time_test)
results['Score_test'].append(score_test)

_times.append(time.time())
checkTime(TMAX_EVAL, "Evaluation des modèles")

# Affichage des résultats
# Display results
df = pandas.DataFrame(results)
display.display(df)

Téléchargement du jeu de données Pendigits...
Téléchargement terminé.


,Classifiers,Optimal_hp1,Optimal_hp2,Time_train,Time_test,Score_train,Score_test
0,KNeighborsClassifier,uniform,1.00000,25.199426,1.025696,0.988545,0.987436
1,SVC,10,1.00000,107.368941,0.301947,0.995091,0.994173
2,MLPClassifier,"(75,)",0.00002,165.199040,0.032011,0.965273,0.967953
